<a href="https://colab.research.google.com/github/ChiroDeniro/ai-ml-projecten/blob/main/GUI_Factuurchecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr

!pip install -q PyPDF2 pdfplumber pdf2image pytesseract gradio==4.44.0



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB

In [2]:
import PyPDF2
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

print("Alles OK!")


Alles OK!


In [ ]:
import os
import re
import tempfile
import PyPDF2
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import gradio as gr

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# ---------- Extract methods ----------
def extract_pypdf2(path):
    try:
        reader = PyPDF2.PdfReader(path)
        return "\n".join(page.extract_text() or "" for page in reader.pages)
    except:
        return ""

def extract_plumber(path):
    try:
        text = ""
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
                text += "\n"
        return text
    except:
        return ""

def extract_ocr(path):
    try:
        imgs = convert_from_path(path, dpi=300)
        return "\n".join(pytesseract.image_to_string(img) for img in imgs)
    except:
        return ""

# ---------- Smart field extraction ----------
def extract_fields(text):
    out = {
        "factuurnummer": None,
        "datum": None,
        "totaal": None,
        "verkoper_blok": None,
        "klant_blok": None,
    }

    # Factuurnummer
    m = re.search(r"factuurnummer[:\s]*([0-9A-Za-z\-]+)", text, re.I)
    if m: out["factuurnummer"] = m.group(1)

    # Datum (eerste datum)
    m = re.search(r"([0-9]{2}[-/][0-9]{2}[-/][0-9]{4})", text)
    if m: out["datum"] = m.group(1)

    # Totaalbedrag
    m = re.search(r"totaal[^0-9]*([0-9]+[\.,][0-9]{2})", text, re.I)
    if m: out["totaal"] = m.group(1)

    # Postcode-blokken (NL)
    lines = text.split("\n")
    pc = r"\b[0-9]{4}\s?[A-Z]{2}\b"
    blocks = []
    for i,l in enumerate(lines):
        if re.search(pc, l):
            block = "\n".join(lines[max(0,i-3): i+3])
            blocks.append(block)

    if len(blocks) >= 1:
        out["verkoper_blok"] = blocks[0]
    if len(blocks) >= 2:
        out["klant_blok"] = blocks[1]

    return out

# ---------- Handle upload ----------
def process_one(upload, force_ocr):
    if upload is None:
        return {"error": "Geen bestand."}, "", ""

    # Gradio file always has .name
    path = upload.name

    t1 = extract_pypdf2(path)
    t2 = extract_plumber(path)
    t3 = extract_ocr(path) if force_ocr or (len(t1) < 50 and len(t2) < 50) else ""

    combined = "\n".join([t1,t2,t3])
    fields = extract_fields(combined)

    return fields, t3, combined


# ---------- GUI ----------
with gr.Blocks() as gui:
    gr.Markdown("## Factuur Checker – OCR + Structuur Extractie")

    file = gr.File(label="Upload PDF", file_types=[".pdf"])
    force = gr.Checkbox(label="Forceer OCR")
    btn = gr.Button("Verwerk")

    fields_out = gr.JSON(label="Gevonden velden")
    ocr_out = gr.Textbox(label="OCR tekst", lines=10)
    raw_out = gr.Textbox(label="Gecombineerde tekst", lines=10)

    btn.click(process_one, inputs=[file, force], outputs=[fields_out, ocr_out, raw_out])


gui.launch(share=True)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2dae5f00c141134d23.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Streaming output truncated to the last 5000 lines.
               ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/routing.py", line 391, in app
    raw_response = await run_endpoint_function(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/routing.py", line 292, in run_endpoint_function
    return await run_in_threadpool(dependant.call, **values)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/starlette/concurrency.py", line 38, in run_in_threadpool
    return await anyio.to_thread.run_sync(func)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/_backends/_asyncio.py", line 2485, i